# import the packages

In [ ]:
# Add the path of the PyOTIC Software to the system path
import sys
sys.path.append('../..')
sys.path.append('../../../PyOTIC/pyoti/')
#Load investigator module
import pyoti
pyoti.info()
import pyotc

### create experiment and set data path

In [ ]:
# Import os to easily join names of filepaths
import os
# Choose the path, were the experiment should be created (or opened from)
#
# datadir: The path to where the experiment (and the data) are located
# datafile: The name of the file that contains the data. Here it is only used to generate dbfile.
#           The data is loaded further down upon creation of a Record.
# dbfile: The name of the database file the experiment is saved to (or loaded from).

datadir = '../exampleData/height_calibration_time_series/'
datafile = 'B01.bin'

# For the name of the experiment, exchange the extension '.bin' with '.fs'
dbfile = os.path.join(datadir, datafile.replace('.bin', '.fs'))

# Create the experiment (this will implicitly create/open the dbfile in datadir)
# and assign it to the variable 'experiment'
experiment = pyoti.open_experiment(dbfile)

cfgfile = '../../../PyOTIC/pyoti/pyoti/etc/record/ASWAD.cfg'

record = experiment.create_record(filename=datafile,  # data file to read
                                  cfgfile=cfgfile,  # setup specific configuration file
                                  directory=datadir)  # data directory

In [ ]:
experiment._grs.displayrate = 50
experiment.add_view('psd1', 'alpha', traces=['psdXYZ', 'positionXYZ'])

### determine amount of data (time) needed 

In [ ]:
N_avg = 80
ex_freq = 32.0

# posible minimum frequencies
print([ex_freq/ndiv for ndiv in range(1, 6)])

print('choose the minimum frequency:')
min_freq = float(input())

t_psd = 1/min_freq
print('Time for one psd: {0:1.3f} s'.format(t_psd))

t_total = N_avg * t_psd
print('Time needed for measurement: {0:1.3f} s'.format(t_total))

In [ ]:
from pyotc import gen_psdm_from_region

region = experiment.view('psd1')

In [ ]:
T_msr = 5.0
T_delay = 0.5
N_avg = N_avg

es = pyotc.ExpSetting(temp=29.2, radius=0.43,
                      temp_err=0.001, radius_err=0.43e-3,
                      temp_unit='degC', radius_unit='um',
                      height=-1 * region.get_data('positionZ').mean(),
                      height_unit='um',
                      material='ps', medium='water')


# traces to calculate psds of
psd_traces = ['psdX', 'psdY', 'psdZ']

position_unit = 'um'

pm = gen_psdm_from_region(region, T_msr, N_avg, T_delay=1.0, psd_traces=psd_traces, exp_setting=es,
                          active_calibration=True, ex_freq=ex_freq)


In [ ]:
pm.plot_psds()


In [ ]:
pf = pyotc.PSDFit(pm)
pf.setup_fit(model='hydro')

bounds = {'psdX': (40, 15e3), 'psdY': (40, 15e3), 'psdZ': (200, 4e3)}
pf.fit_psds(dynamic_bounds=True, bounds=bounds, plot_fits=True)

In [ ]:
pf.write_results_to_file(directory=pm.directory, fname=pm.paramfile)